In [ ]:
# 📅 Nature Metodolojisi EDA
# 🎯 Nature makalesindeki analizleri Milano verisinde uygulama

print("🚀 NATURE METODOLOJİSİ EDA BAŞLIYOR...")
print("🎯 Hedef: Nature Figure 1-5 benzeri analizler")
print("="*60)

# 1. Kütüphaneleri yükle
!pip install -q google-cloud-bigquery pandas numpy matplotlib seaborn scipy geopandas
print("✅ Kütüphaneler yüklendi")

# 2. Authentication
from google.colab import auth
auth.authenticate_user()
print("✅ Authentication tamamlandı")

# 3. Import'lar
from google.cloud import bigquery
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

print("✅ Tüm import'lar tamamlandı")

In [ ]:
# 📌 HÜCRE 2.1: TABLO SCHEMA KONTROLÜ

print("\n🔍 TABLO SCHEMA KONTROLLERİ")
print("="*60)

def check_table_schema_simple(table_name):
    """Tablonun schema'sını basit şekilde kontrol et"""
    try:
        # Tabloyu direkt okuyarak schema'yı al
        table_ref = client.dataset(DATASET_ID).table(table_name)
        table = client.get_table(table_ref)

        print(f"\n📋 {table_name.upper()} SCHEMA:")
        print(f"   📊 {len(table.schema)} sütun, {table.num_rows:,} satır, {table.num_bytes/(1024*1024):.1f} MB")

        # Sütun bilgilerini göster
        schema_info = []
        for field in table.schema:
            schema_info.append({
                'Column': field.name,
                'Type': field.field_type,
                'Mode': field.mode if hasattr(field, 'mode') else 'NULLABLE'
            })

        df_schema = pd.DataFrame(schema_info)
        print(df_schema.to_string(index=False))

        return df_schema
    except Exception as e:
        print(f"   ⚠️  {table_name}: Schema kontrol edilemedi - {str(e)[:100]}")
        return None

# Nature analizleri için kritik tabloların schema'larını kontrol et
critical_tables = [
    'cell_summary_nature',
    'traffic_winsorized',
    'movement_clean',
    'census_data',
    'temporal_pattern_nature'
]

schemas = {}
print("\n📊 TABLO SCHEMA ÖZETLERİ:")
print("="*60)

for table in critical_tables:
    if table in df_tables['table_name'].values:
        schemas[table] = check_table_schema_simple(table)

# Nature metodolojisi için gerekli sütunları kontrol et
print("\n🎯 NATURE METODOLOJİSİ GEREKSİNİMLERİ KONTROLÜ")
print("="*60)

nature_requirements = {
    'cell_summary_nature': ['CellID', 'total_activity', 'total_smsin', 'total_internet'],
    'traffic_winsorized': ['datetime', 'CellID', 'smsin_winsorized', 'internet_winsorized'],
    'movement_clean': ['datetime', 'CellID', 'provinceName', 'cell2Province_clean'],
    'census_data': ['PROVINCIA'],  # + diğer 134 demografik değişken
    'temporal_pattern_nature': ['hour_of_day', 'day_of_week', 'total_hourly_smsin', 'total_hourly_internet']
}

print("\n🔍 NATURE ANALİZLERİ İÇİN KRİTİK SÜTUN KONTROLÜ:")
all_requirements_met = True

for table, required_cols in nature_requirements.items():
    if table in schemas and schemas[table] is not None:
        available_cols = schemas[table]['Column'].tolist()
        missing_cols = [col for col in required_cols if col not in available_cols]

        if missing_cols:
            print(f"   ⚠️  {table}: {len(missing_cols)} eksik sütun - {missing_cols}")
            all_requirements_met = False
        else:
            print(f"   ✅ {table}: Tüm {len(required_cols)} gerekli sütun mevcut")
            print(f"      📊 Mevcut sütunlar: {len(available_cols)}")
    else:
        print(f"   ❌ {table}: Schema kontrol edilemedi")
        all_requirements_met = False

if all_requirements_met:
    print("\n🎉 TEBRİKLER! Tüm Nature gereksinimleri karşılanıyor!")
else:
    print("\n⚠️  UYARI: Bazı Nature gereksinimleri eksik")

# Nature'daki analizler için özel sütun analizi
print("\n📊 NATURE ANALİZLERİ İÇİN ÖZEL KONTROLLER")
print("="*60)

# 1. TRAFİK VERİSİ: Nature'daki gibi zaman serisi analizi için
if 'traffic_winsorized' in schemas and schemas['traffic_winsorized'] is not None:
    traffic_cols = schemas['traffic_winsorized']['Column'].tolist()
    print(f"\n📱 TRAFİK VERİSİ ANALİZİ (traffic_winsorized):")
    print(f"   ⏰ Zaman serisi: {'datetime' in traffic_cols}")
    print(f"   📍 Mekansal: {'CellID' in traffic_cols}")
    print(f"   📊 Aktivite metrikleri:")
    activity_metrics = [col for col in traffic_cols if any(x in col.lower() for x in ['sms', 'call', 'internet'])]
    for metric in activity_metrics:
        print(f"      • {metric}")

# 2. CENSUS VERİSİ: Nature'daki poverty mapping için
if 'census_data' in schemas and schemas['census_data'] is not None:
    census_cols = schemas['census_data']['Column'].tolist()
    print(f"\n📊 DEMOGRAFİK VERİ ANALİZİ (census_data):")
    print(f"   📍 Eyalet ismi: {'PROVINCIA' in census_cols}")

    # Demografik değişken kategorileri
    pop_vars = [c for c in census_cols if c.startswith('P')]
    edu_vars = [c for c in census_cols if c.startswith('E')]
    other_vars = [c for c in census_cols if c not in pop_vars + edu_vars + ['PROVINCIA']]

    print(f"   👥 Nüfus değişkenleri: {len(pop_vars)}")
    print(f"   🎓 Eğitim değişkenleri: {len(edu_vars)}")
    print(f"   📈 Diğer değişkenler: {len(other_vars)}")

    # İlk 5 örnek
    print(f"   🔍 Örnek nüfus değişkenleri: {pop_vars[:5]}")
    print(f"   🔍 Örnek eğitim değişkenleri: {edu_vars[:5] if edu_vars else 'Yok'}")

# 3. HAREKET VERİSİ: Nature'daki human mobility için
if 'movement_clean' in schemas and schemas['movement_clean'] is not None:
    movement_cols = schemas['movement_clean']['Column'].tolist()
    print(f"\n🗺️  HAREKET VERİSİ ANALİZİ (movement_clean):")
    print(f"   🚶 Hareket yönü: {'cell2Province_clean' in movement_cols} (Milano'dan)")
    print(f"   🚶 Hareket yönü: {'Province2cell_clean' in movement_cols} (Milano'ya)")
    print(f"   📍 Hedef lokasyon: {'provinceName' in movement_cols}")
    print(f"   ⏰ Zaman: {'datetime' in movement_cols}")

# 4. HÜCRE ÖZETİ: Nature'daki anten özeti için
if 'cell_summary_nature' in schemas and schemas['cell_summary_nature'] is not None:
    cell_cols = schemas['cell_summary_nature']['Column'].tolist()
    print(f"\n📡 HÜCRE ÖZET ANALİZİ (cell_summary_nature):")
    print(f"   📍 Hücre ID: {'CellID' in cell_cols}")
    print(f"   📊 Toplam aktivite: {'total_activity' in cell_cols}")

    # Aktivite türleri
    activity_cols = [c for c in cell_cols if 'total_' in c or 'avg_' in c]
    print(f"   📈 Aktivite metrikleri: {len(activity_cols)}")
    for col in activity_cols[:5]:  # İlk 5'ini göster
        print(f"      • {col}")

print("\n" + "="*60)
print("✅ SCHEMA KONTROLLERİ TAMAMLANDI")
print("="*60)
print("🎯 Nature Figure 1-3 analizlerine geçebiliriz!")

In [ ]:
print("\n" + "="*60)
print("🏙️ NATURE FIGURE 1: ŞEHİR HİYERARŞİSİ ANALİZİ")
print("="*60)

def plot_nature_figure1():
    """Nature Figure 1: Power law distribution of traffic"""

    # 1. Veriyi çek
    query = f"""
    WITH ranked_cells AS (
      SELECT
        total_activity,
        ROW_NUMBER() OVER(ORDER BY total_activity DESC) as rank
      FROM `{PROJECT_ID}.{DATASET_ID}.cell_summary_nature`
      WHERE total_activity > 0
    )
    SELECT
      rank,
      total_activity,
      LOG(rank) as log_rank,
      LOG(total_activity) as log_activity
    FROM ranked_cells
    ORDER BY rank
    """

    df = client.query(query).to_dataframe()

    # 2. Power law fitting (Nature'daki α parametresi)
    from scipy.stats import linregress
    slope, intercept, r_value, p_value, std_err = linregress(
        df['log_activity'], df['log_rank']
    )
    alpha = -slope  # Nature'daki α parametresi

    # 3. Nature Figure 1 benzeri grafik
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Sol: Log-log plot (Nature Figure 1a)
    ax1 = axes[0]
    ax1.scatter(df['total_activity'], df['rank'], alpha=0.6, s=10)
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    ax1.set_xlabel('Total Activity (log scale)', fontsize=12)
    ax1.set_ylabel('Rank (log scale)', fontsize=12)
    ax1.set_title('Nature Figure 1a: Rank-Activity Distribution', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)

    # Trend çizgisi ekle
    x_fit = np.logspace(np.log10(df['total_activity'].min()),
                       np.log10(df['total_activity'].max()), 100)
    y_fit = 10**(intercept) * x_fit**(slope)
    ax1.plot(x_fit, y_fit, 'r-', linewidth=2, label=f'Power law: α={alpha:.2f}')
    ax1.legend()

    # Sağ: Histogram (Nature Figure 1b)
    ax2 = axes[1]
    ax2.hist(df['total_activity'], bins=50, log=True, alpha=0.7, edgecolor='black')
    ax2.set_xscale('log')
    ax2.set_xlabel('Total Activity (log scale)', fontsize=12)
    ax2.set_ylabel('Frequency (log scale)', fontsize=12)
    ax2.set_title('Nature Figure 1b: Activity Distribution', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # 4. İstatistiksel sonuçlar
    print("📊 POWER LAW ANALİZ SONUÇLARI:")
    print(f"   📈 Örneklem büyüklüğü: {len(df):,} hücre")
    print(f"   🎯 α parametresi (Nature'da ≈ 1.19): {alpha:.3f}")
    print(f"   📉 Korelasyon katsayısı (r): {r_value:.3f}")
    print(f"   📊 R² değeri: {r_value**2:.3f}")
    print(f"   🔍 p-değeri: {p_value:.4f}")

    # Nature ile karşılaştırma
    nature_alpha = 1.19
    similarity = 100 * (1 - abs(alpha - nature_alpha)/nature_alpha)

    print(f"\n📚 NATURE KARŞILAŞTIRMASI:")
    print(f"   📖 Nature α: {nature_alpha}")
    print(f"   📊 Bizim α: {alpha:.3f}")
    print(f"   📈 Benzerlik: {similarity:.1f}%")

    return df, alpha

# Çalıştır
df_figure1, alpha = plot_nature_figure1()

In [ ]:
# 📌 HÜCRE 4: NATURE FIGURE 2 - HAREKET MODELİ ANALİZİ

print("\n" + "="*60)
print("🚶 NATURE FIGURE 2: İNSAN HAREKETLİLİĞİ ANALİZİ")
print("="*60)

def plot_nature_figure2():
    """Nature Figure 2: Human mobility patterns"""

    # 1. Eyalet hareket verisini çek - POZİTİF DEĞERLER FİLTRELİ
    query = f"""
    WITH province_stats AS (
      SELECT
        p.province_name,
        m.total_from_milano,
        m.total_to_milano,
        m.net_movement,
        ST_DISTANCE(
          ST_CENTROID(ST_GEOGFROMTEXT(p.geometry)),
          (SELECT ST_CENTROID(ST_GEOGFROMTEXT(geometry))
           FROM `{PROJECT_ID}.{DATASET_ID}.province_boundaries`
           WHERE province_name = 'MILANO')
        ) / 1000 as distance_km
      FROM `{PROJECT_ID}.{DATASET_ID}.province_boundaries` p
      JOIN `{PROJECT_ID}.{DATASET_ID}.province_movement_nature` m
        ON p.province_name = m.provinceName
      WHERE p.province_name != 'MILANO'
        AND m.total_from_milano > 0  -- POZİTİF HAREKET OLANLAR
    )
    SELECT *
    FROM province_stats
    ORDER BY total_from_milano DESC
    """

    df_movement = client.query(query).to_dataframe()

    # 2. Pozitif değerleri kontrol et
    print(f"📊 Hareket verisi: {len(df_movement)} eyalet")
    print(f"   🔍 Pozitif hareket değerleri: {sum(df_movement['total_from_milano'] > 0)}")
    print(f"   📏 Mesafe aralığı: {df_movement['distance_km'].min():.1f} - {df_movement['distance_km'].max():.1f} km")

    # 3. Radyal dağılım analizi (POZİTİF DEĞERLER İLE)
    valid_data = df_movement[
        (df_movement['distance_km'] > 0) &
        (df_movement['total_from_milano'] > 0)
    ].copy()

    if len(valid_data) < 5:
        print("⚠️  Yetersiz veri: En az 5 pozitif hareket kaydı gerekli")
        return None, None

    valid_data['log_distance'] = np.log(valid_data['distance_km'])
    valid_data['log_movement'] = np.log(valid_data['total_from_milano'])

    from scipy.stats import linregress
    slope, intercept, r_value, p_value, std_err = linregress(
        valid_data['log_distance'], valid_data['log_movement']
    )
    lambda_param = -slope  # Nature'daki λ parametresi

    # 4. Nature Figure 2 benzeri grafik
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Sol: Distance vs Movement (Nature Figure 2a) - LOG SCALE DÜZELTMESİ
    ax1 = axes[0]

    # Pozitif değerleri filtrele
    plot_data = valid_data.copy()

    # Log scale için minimum değer ayarla
    min_movement = plot_data['total_from_milano'].min()
    min_distance = plot_data['distance_km'].min()

    scatter = ax1.scatter(plot_data['distance_km'], plot_data['total_from_milano'],
                         alpha=0.7, s=80, c=plot_data['net_movement'], cmap='coolwarm',
                         edgecolors='black', linewidth=0.5)

    # Log scale uygula
    ax1.set_xscale('log')
    ax1.set_yscale('log')

    # Eksen limitlerini ayarla (0'dan büyük)
    x_min = max(0.1, min_distance * 0.9)
    x_max = plot_data['distance_km'].max() * 1.1
    y_min = max(0.1, min_movement * 0.9)
    y_max = plot_data['total_from_milano'].max() * 1.1

    ax1.set_xlim(x_min, x_max)
    ax1.set_ylim(y_min, y_max)

    ax1.set_xlabel('Distance from Milano (km, log scale)', fontsize=12)
    ax1.set_ylabel('Movement Volume (log scale)', fontsize=12)
    ax1.set_title('Nature Figure 2a: Distance-Movement Relationship', fontsize=14, fontweight='bold')

    # Trend çizgisi
    x_fit = np.logspace(np.log10(x_min), np.log10(x_max), 100)
    y_fit = np.exp(intercept) * x_fit**(slope)
    ax1.plot(x_fit, y_fit, 'k--', linewidth=2, label=f'λ={lambda_param:.2f}')
    ax1.legend()

    plt.colorbar(scatter, ax=ax1, label='Net Movement')
    ax1.grid(True, alpha=0.3, which='both')

    # Sağ: Movement distribution (Nature Figure 2b)
    ax2 = axes[1]
    top_20 = df_movement.head(20)
    colors = plt.cm.tab20c(np.arange(len(top_20)))
    bars = ax2.barh(top_20['province_name'], top_20['total_from_milano'], color=colors)
    ax2.set_xlabel('Movement Volume from Milano', fontsize=12)
    ax2.set_title('Nature Figure 2b: Top 20 Destination Provinces', fontsize=14, fontweight='bold')
    ax2.invert_yaxis()

    # Değerleri ekle
    for i, (bar, val) in enumerate(zip(bars, top_20['total_from_milano'])):
        ax2.text(val + val*0.01, bar.get_y() + bar.get_height()/2,
                f'{val:,.0f}', va='center', fontsize=9)

    plt.tight_layout()
    plt.show()

    # 5. İstatistiksel sonuçlar
    print("\n📊 HAREKET ANALİZİ SONUÇLARI:")
    print(f"   📈 Analiz edilen eyalet sayısı: {len(valid_data)}")
    print(f"   📏 Mesafe ortalaması: {valid_data['distance_km'].mean():.1f} km")
    print(f"   🚶 Hareket ortalaması: {valid_data['total_from_milano'].mean():,.0f}")
    print(f"   🎯 λ parametresi (Nature'da ≈ 2.06): {lambda_param:.3f}")
    print(f"   📉 Korelasyon katsayısı: {r_value:.3f}")
    print(f"   📊 R² değeri: {r_value**2:.3f}")
    print(f"   🔍 p-değeri: {p_value:.4f}")

    # Nature ile karşılaştırma
    if not np.isnan(lambda_param):
        nature_lambda = 2.06
        similarity = 100 * (1 - abs(lambda_param - nature_lambda)/nature_lambda)

        print(f"\n📚 NATURE KARŞILAŞTIRMASI:")
        print(f"   📖 Nature λ: {nature_lambda}")
        print(f"   📊 Bizim λ: {lambda_param:.3f}")
        print(f"   📈 Benzerlik: {similarity:.1f}%")
    else:
        print("⚠️  λ parametresi hesaplanamadı")

    return df_movement, lambda_param

# Çalıştır
df_figure2, lambda_param = plot_nature_figure2()

In [ ]:
# 📌 HÜCRE 5: NATURE FIGURE 3 - EV-İŞ KONUMU ÇIKARIMI

print("\n" + "="*60)
print("🏠 NATURE FIGURE 3: EV-İŞ KONUMU ÇIKARIMI")
print("="*60)

def plot_nature_figure3():
    """Nature Figure 3: Home-work location inference"""

    # 1. Zaman bazlı trafik pattern'lerini çek
    query = f"""
    SELECT
      hour_of_day,
      day_of_week,
      total_hourly_smsin,
      total_hourly_internet,
      total_hourly_calls,
      active_cells,
      avg_smsin_per_cell
    FROM `{PROJECT_ID}.{DATASET_ID}.temporal_pattern_nature`
    ORDER BY day_of_week, hour_of_day
    """

    df_temporal = client.query(query).to_dataframe()

    # 2. Nature'daki gibi gece (20:00-08:00) ve gündüz (10:00-17:00) pattern'leri
    df_temporal['period'] = 'other'
    df_temporal.loc[df_temporal['hour_of_day'].between(20, 23) |
                   df_temporal['hour_of_day'].between(0, 8), 'period'] = 'night'
    df_temporal.loc[df_temporal['hour_of_day'].between(10, 17), 'period'] = 'day'

    # 3. Nature Figure 3 benzeri grafik
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # 3.1 Günlük pattern (Nature Figure 3a)
    weekday_data = df_temporal[df_temporal['day_of_week'].between(1, 5)]  # Pazartesi-Cuma
    weekend_data = df_temporal[df_temporal['day_of_week'] >= 6]  # Cumartesi-Pazar

    ax1 = axes[0, 0]
    week_avg = weekday_data.groupby('hour_of_day')['total_hourly_smsin'].mean()
    weekend_avg = weekend_data.groupby('hour_of_day')['total_hourly_smsin'].mean()

    ax1.plot(week_avg.index, week_avg.values, 'b-', linewidth=2, label='Weekdays', marker='o')
    ax1.plot(weekend_avg.index, weekend_avg.values, 'r-', linewidth=2, label='Weekend', marker='s')
    ax1.set_xlabel('Hour of Day', fontsize=12)
    ax1.set_ylabel('SMS Activity', fontsize=12)
    ax1.set_title('Nature Figure 3a: Daily Activity Pattern', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.fill_between([20, 24], 0, ax1.get_ylim()[1], alpha=0.1, color='blue', label='Night (20-8)')
    ax1.fill_between([10, 17], 0, ax1.get_ylim()[1], alpha=0.1, color='green', label='Day (10-17)')
    ax1.legend()

    # 3.2 Haftalık pattern (Nature Figure 3b)
    ax2 = axes[0, 1]
    daily_avg = df_temporal.groupby('day_of_week').agg({
        'total_hourly_smsin': 'mean',
        'total_hourly_internet': 'mean',
        'total_hourly_calls': 'mean'
    }).reset_index()

    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    x_pos = np.arange(len(days))
    width = 0.25

    ax2.bar(x_pos - width, daily_avg['total_hourly_smsin'], width, label='SMS', alpha=0.8)
    ax2.bar(x_pos, daily_avg['total_hourly_internet'], width, label='Internet', alpha=0.8)
    ax2.bar(x_pos + width, daily_avg['total_hourly_calls'], width, label='Calls', alpha=0.8)

    ax2.set_xlabel('Day of Week', fontsize=12)
    ax2.set_ylabel('Average Hourly Activity', fontsize=12)
    ax2.set_title('Nature Figure 3b: Weekly Activity Pattern', fontsize=14, fontweight='bold')
    ax2.set_xticks(x_pos)
    ax2.set_xticklabels(days)
    ax2.legend()
    ax2.grid(True, alpha=0.3, axis='y')

    # 3.3 Gece-gündüz aktivite oranı (Nature'daki home-work inference)
    ax3 = axes[1, 0]
    period_ratio = df_temporal.groupby(['day_of_week', 'period'])['total_hourly_smsin'].mean().unstack()

    # 'day' ve 'night' sütunlarının varlığını kontrol et
    if 'day' in period_ratio.columns and 'night' in period_ratio.columns:
        period_ratio['day_night_ratio'] = period_ratio['day'] / period_ratio['night']

        ax3.bar(range(1, 8), period_ratio['day_night_ratio'], color='purple', alpha=0.7)
        ax3.axhline(y=1, color='r', linestyle='--', alpha=0.5, label='Equal day/night')
        ax3.set_xlabel('Day of Week', fontsize=12)
        ax3.set_ylabel('Day/Night Activity Ratio', fontsize=12)
        ax3.set_title('Nature Figure 3c: Home-Work Inference Metric', fontsize=14, fontweight='bold')
        ax3.set_xticks(range(1, 8))
        ax3.set_xticklabels(days)
        ax3.legend()
        ax3.grid(True, alpha=0.3)
    else:
        ax3.text(0.5, 0.5, 'Day/Night verisi yetersiz',
                ha='center', va='center', transform=ax3.transAxes, fontsize=12)
        ax3.set_title('Nature Figure 3c: Home-Work Inference Metric\n(Veri yetersiz)', fontsize=14, fontweight='bold')

    # 3.4 Aktivite çeşitliliği (Nature'daki activity diversity)
    ax4 = axes[1, 1]
    cell_query = f"""
    SELECT
      CellID,
      total_smsin,
      total_internet,
      total_callin,
      total_callout,
      -- Toplam çağrı = gelen + giden
      (total_callin + total_callout) as total_calls,
      CASE
        WHEN total_smsin > (total_callin + total_callout)
             AND total_smsin > total_internet THEN 'SMS Dominant'
        WHEN total_internet > (total_callin + total_callout)
             AND total_internet > total_smsin THEN 'Internet Dominant'
        WHEN (total_callin + total_callout) > total_smsin
             AND (total_callin + total_callout) > total_internet THEN 'Call Dominant'
        ELSE 'Balanced'
      END as activity_type
    FROM `{PROJECT_ID}.{DATASET_ID}.cell_summary_nature`
    """

    df_cell_types = client.query(cell_query).to_dataframe()
    type_counts = df_cell_types['activity_type'].value_counts()

    colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99']
    wedges, texts, autotexts = ax4.pie(type_counts.values, labels=type_counts.index,
                                       autopct='%1.1f%%', colors=colors, startangle=90)
    ax4.set_title('Nature Figure 3d: Cell Activity Diversity', fontsize=14, fontweight='bold')

    plt.tight_layout()
    plt.show()

    # 4. İstatistiksel analiz
    print("📊 EV-İŞ KONUMU ANALİZİ SONUÇLARI:")

    # Period ratio analizi
    if 'day' in period_ratio.columns and 'night' in period_ratio.columns:
        print(f"   ☀️  Gündüz aktivite ortalaması: {period_ratio['day'].mean():.2f}")
        print(f"   🌙 Gece aktivite ortalaması: {period_ratio['night'].mean():.2f}")
        print(f"   📊 Gündüz/Gece oranı: {period_ratio['day_night_ratio'].mean():.2f}")

        # Nature'daki "home-work location inference" ile benzerlik
        print(f"\n📚 NATURE BENZERLİĞİ:")
        print("   🔍 Nature'da gündüz aktivitesi iş yerlerini gösterir")
        print("   🔍 Nature'da gece aktivitesi evleri gösterir")
        print("   📊 Bizim verimizde gündüz aktivitesi {:.1f}x daha fazla".format(
            period_ratio['day_night_ratio'].mean()))
    else:
        print("   ⚠️  Gün/Gece analizi için yeterli veri yok")

    # Hücre aktivite dağılımı
    print(f"\n📱 HÜCRE AKTİVİTE DAĞILIMI:")
    for activity_type, count in type_counts.items():
        percentage = 100 * count / len(df_cell_types)
        print(f"   {activity_type}: {count:,} hücre ({percentage:.1f}%)")

    # Zaman serisi özeti
    print(f"\n⏰ ZAMANSAL PATTERN ÖZETİ:")
    print(f"   📅 Gözlem sayısı: {len(df_temporal)} (24 saat × 7 gün)")
    print(f"   📱 Ortalama SMS aktivitesi: {df_temporal['total_hourly_smsin'].mean():.0f}")
    print(f"   🌐 Ortalama internet aktivitesi: {df_temporal['total_hourly_internet'].mean():.0f}")
    print(f"   📞 Ortalama çağrı aktivitesi: {df_temporal['total_hourly_calls'].mean():.0f}")

    return df_temporal, period_ratio if 'day_night_ratio' in period_ratio.columns else None, df_cell_types

# Çalıştır
df_figure3, period_ratio, df_cell_types = plot_nature_figure3()

In [ ]:
print("🔍 FIGURE 2 DEBUG: İSİM UYUŞMAZLIĞI KONTROLÜ")
print("="*60)

# 1. Hareket verisindeki eyalet isimlerini kontrol et
query_mov = f"""
SELECT DISTINCT provinceName
FROM `{PROJECT_ID}.{DATASET_ID}.movement_clean`
WHERE provinceName IS NOT NULL
LIMIT 10
"""
print("📋 Movement Tablosundaki İsimler:")
df_mov_names = client.query(query_mov).to_dataframe()
print(df_mov_names)

# 2. Sınır verisindeki eyalet isimlerini kontrol et
query_bound = f"""
SELECT DISTINCT province_name
FROM `{PROJECT_ID}.{DATASET_ID}.province_boundaries`
LIMIT 10
"""
print("\n📋 Boundaries Tablosundaki İsimler:")
df_bound_names = client.query(query_bound).to_dataframe()
print(df_bound_names)

# 3. Çapraz kontrol (Eşleşmeyenleri bul)
query_check = f"""
SELECT
  m.provinceName as mov_name,
  b.province_name as bound_name,
  COUNT(*) as count
FROM `{PROJECT_ID}.{DATASET_ID}.movement_clean` m
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.province_boundaries` b
  ON UPPER(TRIM(m.provinceName)) = UPPER(TRIM(b.province_name))
GROUP BY 1, 2
LIMIT 10
"""
print("\n⚠️ Eşleşme Kontrolü (Sample):")
try:
    df_check = client.query(query_check).to_dataframe()
    print(df_check)
except Exception as e:
    print(e)

In [ ]:
# Gerekli kütüphaneleri ve ayarları tekrar tanımlayalım
from google.cloud import bigquery
import pandas as pd
import geopandas as gpd
import os
from google.colab import drive
drive.mount('/content/drive')

# BigQuery Client
PROJECT_ID = "YOUR_PROJECT_ID"
DATASET_ID = "milano_mobile_2013"
client = bigquery.Client(project=PROJECT_ID)

# 📂 DOSYA YOLUNU TANIMLIYORUZ (Hatanın sebebi burasıydı)
BASE_PATH = '/content/drive/MyDrive/Milano Telecom Data Analysis — 2013 Week Dataset/archive/'

# Eğer klasör bulunamazsa alternatifleri kontrol et (Güvenlik önlemi)
if not os.path.exists(BASE_PATH):
    print("⚠️ Standart yol bulunamadı, alternatifler aranıyor...")
    possible_paths = [
        '/content/drive/MyDrive/Milano Telecom Data Analysis — 2013 Week Dataset/',
        '/content/drive/MyDrive/archive/',
        '/content/drive/MyDrive/'
    ]
    for path in possible_paths:
        if os.path.exists(os.path.join(path, "Italian_provinces.geojson")):
            BASE_PATH = path
            print(f"✅ Dosyalar şurada bulundu: {BASE_PATH}")
            break

# GEOJSON KOORDİNAT SİSTEMİ DÜZELTME VE YENİDEN YÜKLEME
def fix_and_reload_geojsons():
    print("🔧 GEOJSON DÜZELTME VE YÜKLEME BAŞLIYOR...")
    print("="*60)

    # 1. İTALYA EYALETLERİ
    print("\n🇮🇹 İtalya Eyaletleri (province_boundaries) düzeltiliyor...")
    provinces_path = os.path.join(BASE_PATH, "Italian_provinces.geojson")

    if os.path.exists(provinces_path):
        try:
            # Dosyayı oku
            gdf_prov = gpd.read_file(provinces_path)
            print(f"   Orijinal CRS: {gdf_prov.crs}")

            # WGS84'e (EPSG:4326) dönüştür
            if gdf_prov.crs != "EPSG:4326":
                gdf_prov = gdf_prov.to_crs("EPSG:4326")
                print(f"   ✅ EPSG:4326 (Lat/Lon) formatına dönüştürüldü")

            # Sütun adlarını bul
            name_col = next((col for col in ['PROVINCIA', 'name', 'NAME'] if col in gdf_prov.columns), None)

            # DataFrame'e çevir (WKT formatında geometri ile)
            df_prov = pd.DataFrame({
                'province_name': gdf_prov[name_col] if name_col else gdf_prov.index.astype(str),
                'geometry': gdf_prov.geometry.to_wkt()
            })

            # BigQuery'e Yükle
            table_id = f"{PROJECT_ID}.{DATASET_ID}.province_boundaries"
            job_config = bigquery.LoadJobConfig(
                write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
            )
            job = client.load_table_from_dataframe(df_prov, table_id, job_config=job_config)
            job.result()
            print(f"   ✅ province_boundaries yeniden yüklendi ({len(df_prov)} satır)")

        except Exception as e:
            print(f"   ❌ Hata: {e}")
    else:
        print(f"   ❌ Dosya bulunamadı: {provinces_path}")

    # 2. MILANO GRID
    print("\n📍 Milano Grid (grid_locations) düzeltiliyor...")
    grid_path = os.path.join(BASE_PATH, "milano-grid.geojson")

    if os.path.exists(grid_path):
        try:
            gdf_grid = gpd.read_file(grid_path)
            print(f"   Orijinal CRS: {gdf_grid.crs}")

            if gdf_grid.crs != "EPSG:4326":
                gdf_grid = gdf_grid.to_crs("EPSG:4326")
                print(f"   ✅ EPSG:4326 (Lat/Lon) formatına dönüştürüldü")

            # ID sütununu bul
            id_col = next((col for col in ['cellId', 'CellID', 'id'] if col in gdf_grid.columns), None)

            df_grid = pd.DataFrame({
                'CellID': gdf_grid[id_col] if id_col else range(1, len(gdf_grid)+1),
                'geometry': gdf_grid.geometry.to_wkt()
            })

            table_id = f"{PROJECT_ID}.{DATASET_ID}.grid_locations"
            job_config = bigquery.LoadJobConfig(
                write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
            )
            job = client.load_table_from_dataframe(df_grid, table_id, job_config=job_config)
            job.result()
            print(f"   ✅ grid_locations yeniden yüklendi ({len(df_grid)} satır)")

        except Exception as e:
            print(f"   ❌ Hata: {e}")
    else:
        print(f"   ❌ Dosya bulunamadı: {grid_path}")

# Düzeltme fonksiyonunu çalıştır
fix_and_reload_geojsons()

In [ ]:
# Gerekli kütüphaneleri tekrar import edelim (garanti olsun)
from google.cloud import bigquery
import pandas as pd
import geopandas as gpd
import os
import shapely.wkt

# BigQuery Client
client = bigquery.Client(project=PROJECT_ID)

# GEOJSON ONARMA VE YÜKLEME FONKSİYONU
def fix_and_reload_geojsons_v2():
    print("🔧 GEOJSON GEOMETRİ ONARMA VE YÜKLEME BAŞLIYOR...")
    print("="*60)

    # 1. İTALYA EYALETLERİ (Province Boundaries)
    print("\n🇮🇹 İtalya Eyaletleri (province_boundaries) onarılıyor...")
    provinces_path = os.path.join(BASE_PATH, "Italian_provinces.geojson")

    if os.path.exists(provinces_path):
        try:
            # Okuma
            gdf_prov = gpd.read_file(provinces_path)

            # 1. CRS Dönüşümü
            if gdf_prov.crs != "EPSG:4326":
                gdf_prov = gdf_prov.to_crs("EPSG:4326")

            # 2. KRİTİK DÜZELTME: Geometriyi temizle (buffer(0))
            # Bu işlem self-intersection ve duplicate vertex hatalarını düzeltir
            gdf_prov['geometry'] = gdf_prov['geometry'].buffer(0)
            print("   ✅ Geometri topolojisi onarıldı (buffer(0) uygulandı)")

            # Sütun adı belirleme
            name_col = next((col for col in ['PROVINCIA', 'name', 'NAME'] if col in gdf_prov.columns), None)

            # DataFrame hazırlığı
            df_prov = pd.DataFrame({
                'province_name': gdf_prov[name_col] if name_col else gdf_prov.index.astype(str),
                'geometry': gdf_prov.geometry.to_wkt()
            })

            # Yükleme
            table_id = f"{PROJECT_ID}.{DATASET_ID}.province_boundaries"
            job_config = bigquery.LoadJobConfig(
                write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
            )
            job = client.load_table_from_dataframe(df_prov, table_id, job_config=job_config)
            job.result()
            print(f"   ✅ province_boundaries başarıyla yüklendi ({len(df_prov)} satır)")

        except Exception as e:
            print(f"   ❌ Hata: {e}")

    # 2. MILANO GRID (Grid Locations)
    print("\n📍 Milano Grid (grid_locations) onarılıyor...")
    grid_path = os.path.join(BASE_PATH, "milano-grid.geojson")

    if os.path.exists(grid_path):
        try:
            # Okuma
            gdf_grid = gpd.read_file(grid_path)

            # 1. CRS Dönüşümü
            if gdf_grid.crs != "EPSG:4326":
                gdf_grid = gdf_grid.to_crs("EPSG:4326")

            # 2. KRİTİK DÜZELTME
            gdf_grid['geometry'] = gdf_grid['geometry'].buffer(0)
            print("   ✅ Geometri topolojisi onarıldı")

            # ID sütunu
            id_col = next((col for col in ['cellId', 'CellID', 'id'] if col in gdf_grid.columns), None)

            # DataFrame
            df_grid = pd.DataFrame({
                'CellID': gdf_grid[id_col] if id_col else range(1, len(gdf_grid)+1),
                'geometry': gdf_grid.geometry.to_wkt()
            })

            # Yükleme
            table_id = f"{PROJECT_ID}.{DATASET_ID}.grid_locations"
            job_config = bigquery.LoadJobConfig(
                write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
            )
            job = client.load_table_from_dataframe(df_grid, table_id, job_config=job_config)
            job.result()
            print(f"   ✅ grid_locations başarıyla yüklendi ({len(df_grid)} satır)")

        except Exception as e:
            print(f"   ❌ Hata: {e}")

# Fonksiyonu çalıştır
fix_and_reload_geojsons_v2()

In [ ]:
def plot_nature_figure2_final():
    """Nature Figure 2: Human mobility patterns"""

    print("\n🚶 NATURE FIGURE 2: İNSAN HAREKETLİLİĞİ ANALİZİ")
    print("="*60)

    # DÜZELTME:
    # 1. province_movement_nature tablosunda 'cell2Province_clean' yok, 'total_from_milano' var.
    # 2. Gereksiz GROUP BY kaldırıldı (tablo zaten gruplanmış durumda).

    query = f"""
    WITH province_stats AS (
      SELECT
        p.province_name,
        m.total_from_milano,
        ST_DISTANCE(
          ST_CENTROID(ST_GEOGFROMTEXT(p.geometry)),
          (SELECT ST_CENTROID(ST_GEOGFROMTEXT(geometry))
           FROM `{PROJECT_ID}.{DATASET_ID}.province_boundaries`
           WHERE UPPER(province_name) = 'MILANO')
        ) / 1000 as distance_km
      FROM `{PROJECT_ID}.{DATASET_ID}.province_boundaries` p
      JOIN `{PROJECT_ID}.{DATASET_ID}.province_movement_nature` m
        -- İSİM EŞLEŞTİRME (Büyük/Küçük harf duyarsız):
        ON UPPER(TRIM(p.province_name)) = UPPER(TRIM(m.provinceName))
      WHERE UPPER(p.province_name) != 'MILANO'
    )
    SELECT *
    FROM province_stats
    WHERE total_from_milano > 0 AND distance_km > 0
    ORDER BY total_from_milano DESC
    """

    try:
        df_movement = client.query(query).to_dataframe()
        print(f"✅ Eşleşen ve verisi olan eyalet sayısı: {len(df_movement)}")

        if len(df_movement) < 5:
            print("❌ Yetersiz veri. Eşleşme veya veri sorunu devam ediyor.")
            print("   Debug: province_movement_nature tablosunu kontrol edin.")
            return None

        # İstatistiksel Analiz (Power Law Fitting)
        df_movement['log_distance'] = np.log(df_movement['distance_km'])
        df_movement['log_movement'] = np.log(df_movement['total_from_milano'])

        from scipy.stats import linregress
        slope, intercept, r_value, p_value, std_err = linregress(
            df_movement['log_distance'], df_movement['log_movement']
        )
        lambda_param = -slope

        print(f"📊 ANALİZ SONUÇLARI:")
        print(f"   🎯 λ parametresi (Nature ≈ 2.06): {lambda_param:.3f}")
        print(f"   📉 R² değeri: {r_value**2:.3f}")
        print(f"   📏 Ortalama Mesafe: {df_movement['distance_km'].mean():.1f} km")

        # Görselleştirme
        fig, axes = plt.subplots(1, 2, figsize=(18, 6))

        # 1. Mesafe vs Hareket (Log-Log)
        axes[0].scatter(df_movement['distance_km'], df_movement['total_from_milano'],
                       alpha=0.6, s=80, c=df_movement['distance_km'], cmap='viridis')
        axes[0].set_xscale('log')
        axes[0].set_yscale('log')
        axes[0].set_xlabel('Mesafe (km) [Log]', fontsize=12)
        axes[0].set_ylabel('Hareket Hacmi [Log]', fontsize=12)
        axes[0].set_title(f'Mesafe Bozunumu (Distance Decay)\nλ = {lambda_param:.2f}', fontsize=14)

        # Trend çizgisi
        x_vals = np.logspace(np.log10(df_movement['distance_km'].min()),
                            np.log10(df_movement['distance_km'].max()), 100)
        y_vals = np.exp(intercept) * x_vals**(slope)
        axes[0].plot(x_vals, y_vals, 'r--', linewidth=2, label='Fit')
        axes[0].grid(True, alpha=0.3)
        axes[0].legend()

        # 2. En Çok Etkileşim Olan Eyaletler
        top_15 = df_movement.head(15)
        sns.barplot(x='total_from_milano', y='province_name', data=top_15, ax=axes[1], palette='Blues_d')
        axes[1].set_title('Milano ile En Çok Etkileşen 15 Eyalet', fontsize=14)
        axes[1].set_xlabel('Toplam Hareket', fontsize=12)
        axes[1].set_ylabel('')

        plt.tight_layout()
        plt.show()

        return df_movement

    except Exception as e:
        print(f"❌ Hata oluştu: {e}")
        return None

# Düzeltilmiş analizi çalıştır
df_figure2 = plot_nature_figure2_final()

In [ ]:
def plot_nature_figure4_network_revised(df_movement_data):
    """Nature Figure 4: Spatial Network Analysis (REVİZE EDİLMİŞ)"""
    print("\n🌐 NATURE FIGURE 4: MEKANSAL ETKİLEŞİM AĞI")
    print("="*60)

    import networkx as nx
    import matplotlib.pyplot as plt
    import numpy as np

    # Veri Kontrolü
    if df_movement_data is None or df_movement_data.empty:
        print("❌ Hata: Figure 2 verisi boş. Lütfen önce Figure 2'yi başarıyla çalıştırın.")
        return

    # Veri Hazırlığı (Sadece en güçlü 50 bağlantıyı alalım ki grafik karışmasın)
    df_net = df_movement_data.head(50).copy()

    # Graph Oluşturma
    G = nx.Graph()

    # Merkez Düğüm
    center_node = "MILANO"
    G.add_node(center_node, size=5000, color='red', label=center_node)

    # Normalizasyon faktörleri
    max_flow = df_net['total_from_milano'].max()

    print(f"📊 Ağ oluşturuluyor... ({len(df_net)} bağlantı)")

    for _, row in df_net.iterrows():
        province = row['province_name']
        flow = row['total_from_milano']
        dist = row['distance_km']

        # Düğüm ekle (Boyut harekete göre)
        node_size = (flow / max_flow) * 3000 + 100
        G.add_node(province, size=node_size, color='skyblue', label=province)

        # Kenar ekle (Ağırlık harekete göre, Uzunluk mesafeye göre)
        # NetworkX spring layout için weight ne kadar büyükse o kadar yakın çeker
        weight = (flow / max_flow) * 10
        G.add_edge(center_node, province, weight=weight, distance=dist)

    # Görselleştirme
    plt.figure(figsize=(20, 15))

    # Yerleşim (En çok etkileşim olanlar merkeze yakın olsun)
    pos = nx.spring_layout(G, k=0.3, iterations=50, weight='weight')

    # Düğümleri çiz
    node_sizes = [nx.get_node_attributes(G, 'size')[node] for node in G.nodes()]
    node_colors = [nx.get_node_attributes(G, 'color')[node] for node in G.nodes()]

    nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, alpha=0.9, edgecolors='black')

    # Kenarları çiz
    edges = G.edges()
    weights = [G[u][v]['weight'] for u,v in edges]
    nx.draw_networkx_edges(G, pos, width=weights, alpha=0.3, edge_color='gray')

    # Etiketleri çiz (Sadece büyük düğümler için)
    labels = {}
    for node in G.nodes():
        if G.nodes[node]['size'] > 500 or node == center_node:
            labels[node] = node

    nx.draw_networkx_labels(G, pos, labels, font_size=10, font_weight='bold')

    plt.title("Milano Merkezli Hareketlilik Ağı (Top 50 İl)", fontsize=20)
    plt.axis('off')
    plt.show()

    print("✅ Ağ analizi tamamlandı.")

# Çalıştırmak için Figure 2 sonucunu kullanıyoruz
# df_figure2'nin tanımlı olduğundan emin olun
if 'df_figure2' in locals():
    plot_nature_figure4_network_revised(df_figure2)
else:
    print("⚠️ Lütfen önce Figure 2 kodunu çalıştırın.")

In [ ]:
def plot_nature_figure5_socioeconomic_revised():
    """Nature Figure 5: Socio-economic correlations (REVİZE EDİLMİŞ SQL)"""
    print("\n💰 NATURE FIGURE 5: SOSYO-EKONOMİK İLİŞKİLER")
    print("="*60)

    # DÜZELTME NOTLARI:
    # 1. census_data tablosunda 'PROVINCIA' sütunu var.
    # 2. province_movement_nature tablosunda 'provinceName' ve 'total_from_milano' var.
    # 3. İki tabloyu UPPER ve TRIM ile eşleştiriyoruz.

    query = f"""
    WITH census_agg AS (
      SELECT
        UPPER(TRIM(PROVINCIA)) as join_key,
        PROVINCIA as original_name,
        -- P1: Toplam Nüfus (Genellikle Census verilerinde P1 nüfustur)
        SUM(P1) as population,
        -- E27: Toplam İstihdam/Ekonomik Aktivite (Varsayılan)
        SUM(E27) as economic_indicator
      FROM `{PROJECT_ID}.{DATASET_ID}.census_data`
      GROUP BY 1, 2
    ),
    movement_agg AS (
      SELECT
        UPPER(TRIM(provinceName)) as join_key,
        SUM(total_from_milano) as total_interaction
      FROM `{PROJECT_ID}.{DATASET_ID}.province_movement_nature`
      GROUP BY 1
    )
    SELECT
      c.original_name as province,
      c.population,
      c.economic_indicator,
      m.total_interaction,
      -- Gravity Index: (Popülasyon * Hareket) / 1000
      (c.population * m.total_interaction) / 1000000 as gravity_index
    FROM census_agg c
    JOIN movement_agg m ON c.join_key = m.join_key
    WHERE c.population > 0 AND m.total_interaction > 0
    ORDER BY c.population DESC
    """

    try:
        df_socio = client.query(query).to_dataframe()
        print(f"✅ Eşleşen İl Sayısı: {len(df_socio)}")

        if len(df_socio) < 5:
            print("❌ Yetersiz veri eşleşmesi. İsim formatlarını kontrol edin.")
            return

        # İstatistiksel Analiz
        # Hareket Hacmi ile Nüfus arasındaki korelasyon (Gravity Modelinin Temeli)
        correlation = df_socio['population'].corr(df_socio['total_interaction'])

        # Görselleştirme
        fig, axes = plt.subplots(1, 2, figsize=(18, 6))

        # Grafik 1: Nüfus vs Etkileşim (Log-Log)
        # Gravity modeline göre nüfus arttıkça etkileşim artmalı
        axes[0].scatter(df_socio['population'], df_socio['total_interaction'],
                       alpha=0.7, c='green', s=60, edgecolors='white')
        axes[0].set_xscale('log')
        axes[0].set_yscale('log')
        axes[0].set_xlabel('İl Nüfusu (Log)', fontsize=12)
        axes[0].set_ylabel('Milano ile Etkileşim (Log)', fontsize=12)
        axes[0].set_title(f'Gravity Model Testi\nKorelasyon: {correlation:.2f}', fontsize=14)

        # Trend çizgisi
        z = np.polyfit(np.log(df_socio['population']), np.log(df_socio['total_interaction']), 1)
        p = np.poly1d(z)
        x_vals = np.linspace(df_socio['population'].min(), df_socio['population'].max(), 100)
        axes[0].plot(x_vals, np.exp(p(np.log(x_vals))), "r--", linewidth=2, label='Trend')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)

        # Grafik 2: Ekonomik Gösterge vs Etkileşim
        # Ekonomik aktivite arttıkça hareketlilik artıyor mu?
        axes[1].scatter(df_socio['economic_indicator'], df_socio['total_interaction'],
                       alpha=0.7, c='purple', s=60, edgecolors='white')
        axes[1].set_xscale('log')
        axes[1].set_yscale('log')
        axes[1].set_xlabel('Ekonomik Aktivite İndeksi (Log)', fontsize=12)
        axes[1].set_ylabel('Milano ile Etkileşim (Log)', fontsize=12)
        axes[1].set_title('Ekonomik Bağlılık Analizi', fontsize=14)
        axes[1].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

        print("📊 SONUÇLAR:")
        print(f"   • Nüfus-Hareket Korelasyonu: {correlation:.3f}")
        print("   • Yorum: Nüfus büyüklüğü ile Milano'dan gelen trafik arasında güçlü bir ilişki beklenir.")
        print("     Bu, Nature makalesindeki sosyo-ekonomik tahmin modelini destekler.")

    except Exception as e:
        print(f"❌ Hata: {e}")

# Çalıştır
plot_nature_figure5_socioeconomic_revised()